In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
from zipfile import ZipFile
from io import BytesIO

import requests
import pandas as pd
import re

In [ ]:
#define 'sourceURL' as the target page that contains the list of zipped csv files
sourceURL = "https://s3.amazonaws.com/capitalbikeshare-data/index.html"

#call chrome webdriver from a local source as 'driver'
#requires chrome driver from https://sites.google.com/a/chromium.org/chromedriver/downloads
#Must change path to location of your chrome driver
driver = webdriver.Chrome('/Users/cheese/lData/chromedriver')  

#use webdriver to call the URL
#Target URL launches in Chrome browser controlled by automated test software
driver.get(sourceURL)

#use WebDriverWait to make sure the page loads before moving on, check by table visibility
wait = WebDriverWait(driver, 10)
element = wait.until(EC.visibility_of_all_elements_located((By.CLASS_NAME, "table")))
#ref1: https://github.com/SeleniumHQ/selenium/blob/master/py/test/selenium/webdriver/common/webdriverwait_tests.py
#ref2: https://selenium-python.readthedocs.io/api.html

#pass page to beautiful soupe as 'soup'
soup = BeautifulSoup(driver.page_source, 'lxml')

#scrape links from beautiful soup in 'links'
links = []

for link in soup.findAll('a', attrs={'href': re.compile("https://")}):
        links.append(link.get('href'))
        
#confirm by viewing all entires in the 'link' array
print(links)

In [ ]:
df = pd.DataFrame() #create a new dataframe as 'df' that will contain all rows from all csv extractions

for fileURL in links: #iterate over each stored URL in the 'link' array
    if fileURL.endswith('.zip'): #Only process URLs ending in .zip to skip scraped links such as index.html
        content = requests.get(fileURL) #use requests.get to store each URL's content as 'content'
        zf = ZipFile(BytesIO(content.content)) #use ZipFile to open and read zip data directly with BytesIO
    #Use the zip file name to identify only the csv files for extraction that match the zip file name   
        for item in zf.namelist():
            print("File in zip: "+ item)
            #printing Filematch index to troubleshoot index out of range issue
            print("Filematch index: " + fileMatch)
        fileMatch = [s for s in zf.namelist() if ".csv" in s][0]
        
        #for the current iteration where the csv file name matches, extract csv from zip, and read into a dataframe 'df_n'
        df_n = pd.read_csv(zf.open(fileMatch), low_memory=False) #df_n will be overwritten with each iteration
        df = df.append(df_n) #Append each new iteration's dataframe from 'df_n' to 'df'
#all file names should display in the order that they are processed

In [ ]:
#print the dataframe 'df' that contains all data from Cap Bike Share
df